# Global Warming visualizer by Cellular Automata 

Nadav Meidan

In [9]:
import itertools
try:
    import tkinter
except ImportError:
    import Tkinter as tk
import random


In [10]:
#Global arrays to collect the data.

TEMPERATURE_STATS = []
AIR_POLLUTION_STATS = []
DAILY_TEMP_STATS = []
DAILY_POLLUTION_STATS = []


In [11]:
class Cell:
    def __init__(self, table, x, y, type_, temperature=25, wind_speed=0, wind_direction="N", cloud_precipitation=0, airPollution=0, airPollutionFactor=0.1):
        self.table = table
        self.x = x
        self.y = y

        self.type = type_  # one of: land, urban, clouds, forest, sea, iceberg
        self.temperature = temperature  # in celsius
        self.wind_speed = wind_speed  # in km/h
        self.wind_direction = wind_direction  # one of: N, S, W, E
        self.cloud_precipitation = cloud_precipitation  # when this reaches 1, it'll start raining
        self.airPollution = airPollution  # 0 is not polluted at all, 1 is fully polluted
        self.airPollutionFactor = airPollutionFactor

        self.baseTemp = temperature

        # temp vars, for commiting the data 
        self.tempType = self.type
        self.tempTemperature = self.temperature
        self.tempWindSpeed = self.wind_speed
        self.tempWindDirection = self.wind_direction
        self.tempCloudPrecipitation = self.cloud_precipitation
        self.tempAirPollution = self.airPollution

    def __repr__(self):
        return "<Cell<{},{}> object, Temp: {}>".format(self.x, self.y, self.temperature)

    def commit(self):
        self.type = self.tempType
        self.temperature = self.tempTemperature
        self.wind_speed =  self.tempWindSpeed
        self.wind_direction = self.tempWindDirection
        self.cloud_precipitation = self.tempCloudPrecipitation
        self.airPollution = self.tempAirPollution

    def calcTheNextDay(self): #this function shows the effects of one day on another
        # before wind
        if self.airPollution >= 0.3:
            # air polution is critical so temprature goes up
            self.tempTemperature = self.temperature + (self.airPollution / 2)
        else:
            # air pollution is low,  we should go slowly towards the baseTemp.
            if (self.temperature - self.baseTemp) > 0:
                # if we're above the baseTemp, we go down
                self.tempTemperature = self.temperature - 0.5
            elif (self.temperature - self.baseTemp) < 0:
                # if we're below the baseTemp, we go up
                self.tempTemperature = self.temperature + 0.5

        if self.type == 'urban':
            # urban areas creates air pollution
            self.tempAirPollution += self.airPollutionFactor
            self._normalize('tempAirPollution')
            
        if self.type == 'clouds':
            if self.temperature >= 30:
                # tropical whether of rain forests
                self.tempType = 'forest'

        if self.type == 'forest':
            # when a forest hits air polution of 1.0, it dies, and becomes a dry land
            if self.tempAirPollution >= 1.0:
                self.tempType = 'land'
            else:
                # if a forest lives, it releases oxygen which helps clean the air polution
                self.tempAirPollution -= 0.1
                self._normalize('tempAirPollution')

        if self.type in ['forest', 'urban']:
            if self.temperature > 70:
                # everything dies at 70 degrees
                self.tempType = 'land'

        if self.type == "sea":
            if self.temperature > 100:
                # water evaporates at 100 degrees
                self.tempType = 'land'

        if self.type == 'iceberg':
            if self.temperature > 0:
                # icebergs melt at positive degrees
                self.tempType = 'sea'
        
        # if it's raining
        if self.cloud_precipitation >= 1.0:
            # if it's raining, it's clears some of the air polution, and decreases the temprature by a little bit.
            self.tempAirPollution /=  1.5
            self._normalize('tempAirPollution')
            if self.tempTemperature > 10:
                self.tempTemperature -= 0.5
            # the cloud is now empty
            self.tempCloudPrecipitation = 0
        
        elif random.random() > 0.5:
            # if it's not raining, there's a 50% chance the cloud will gain 10% rain precipitation.
            self.tempCloudPrecipitation += 0.1
        
        # wind
        wind_neighbours = self.get_incoming_wind_neighbours()
        if wind_neighbours:
            totalPollution = 0.0
            totalWind = 0
            totalRelevantNeighbour = 0
            for neighbour in wind_neighbours:
                # the air pollution gets sucked to here
                if neighbour.airPollution > 0:
                    totalPollution += neighbour.airPollution
                    totalRelevantNeighbour += 1
                    neighbour.tempAirPollution = 0

                totalWind += neighbour.wind_speed
            if totalRelevantNeighbour > 0:
                self.tempAirPollution += totalPollution * 1.0 /totalRelevantNeighbour
            
            self.tempWindSpeed = totalWind / len(wind_neighbours)
            self._normalize('tempAirPollution')

        # there's a 30% chance the wind will change its course
        if random.random() > 0.7:
            self.tempWindDirection = "N" if self.tempWindDirection == "W" else "W"

    def getNeighbour(self, direction=None):
        return self.table.getNeighbour(self.x, self.y, direction)

    def getNeighbours(self):
        return self.table.getNeighbours(self.x, self.y)

    def get_incoming_wind_neighbours(self, wind_factor=10):
        
        radius1_neighbours = [cell for cell in self.getNeighbours() if cell.wind_speed > 0 and cell.getNeighbour(direction=cell.wind_direction) == self]
        radius2_neighbours = [cell for cell in list(itertools.chain(*[x.getNeighbours() for x in radius1_neighbours])) if cell.wind_speed > wind_factor * 1 and cell.getNeighbour(direction=cell.wind_direction) in radius1_neighbours]
        radius3_neighbours = [cell for cell in list(itertools.chain(*[x.getNeighbours() for x in radius2_neighbours])) if cell.wind_speed > wind_factor * 2 and cell.getNeighbour(direction=cell.wind_direction) in radius2_neighbours]

        totalNeighbours = radius1_neighbours + radius2_neighbours + radius3_neighbours

        return totalNeighbours

    def get_color(self):
        if self.type == "land":
            return "#bab354"
        if self.type == "urban":
            return "#ff2a00"
        if self.type == "clouds":
            return "#666967"
        if self.type == "forest":
            return "#115c2a"
        if self.type == "sea":
            return "#2a65fa"
        if self.type == "iceberg":
            return "#e6e6e6"

    def _normalize(self, var):
        v = getattr(self, var)
        if v > 1:
            v = 1
        elif v < 0:
            v= 0
        
        setattr(self, var, v)


In [12]:
class Table:
    def __init__(self, length, cell_size, map_file, airPollutionFactor):
        self.length = length
        self.map_file = map_file
        self.airPollutionFactor = airPollutionFactor
        self.cells = self._createCells()
        self.day = 0

    def getNeighbours(self, x, y, direction=None):
        l = []

        if not direction or direction == 'E':
            if x == (self.length-1):
                # if we're at the border, we will overlap
                l.append(self.cells[0][y])
            else:
                l.append(self.cells[(x+1) % self.length][y])
        if not direction or direction == 'S':
            if y == (self.length):
                # if we're at the border, we will overlap
                l.append(self.cells[x][0])
            else:
                l.append(self.cells[x][(y+1) % self.length])
        if not direction or direction == 'W':
            if x == 0:
                # if we're at the border, we will overlap
                l.append(self.cells[self.length-1][y])
            else:
                l.append(self.cells[(x-1) % self.length][y])
        if not direction or direction == 'N':
            if y == 0:
                # if we're at the border, we will overlap
                l.append(self.cells[x][self.length-1])
            else:
                l.append(self.cells[x][(y-1) % self.length])

        return l

    def getNeighbour(self, x, y, direction):
        t = self.getNeighbours(x, y, direction)
        if not t:
            return None

        return t[0]

    def runNewDay(self):
        self.day += 1

        t_stat_temp = 0
        t_statempAirPollution = 0

        for x in range(self.length):
            for y in range(self.length):
                self.cells[x][y].calcTheNextDay()

                TEMPERATURE_STATS.append(self.cells[x][y].temperature)
                AIR_POLLUTION_STATS.append(self.cells[x][y].airPollution)
                t_stat_temp += self.cells[x][y].temperature
                t_statempAirPollution += self.cells[x][y].airPollution

        for x in range(self.length):
            for y in range(self.length):
                self.cells[x][y].commit()

        DAILY_TEMP_STATS.append((t_stat_temp * 1.0) / (self.length**2))
        DAILY_POLLUTION_STATS.append((t_statempAirPollution * 1.0) / (self.length**2))


    def _createCells(self):
        # creates an empty self.length * self.length matrix
        cells = [[0]*self.length for i in range(self.length)]
        map = self._load_map(self.map_file)

        for x in range(self.length):
            for y in range(self.length):
                # calculating wind_speed.
                # The distribution is as follows:
                # 15% - no wind
                # 40% - 1-10 km/h wind
                # 40% - 11-20 km/h wind
                # 15% - 21-30 km/h wind

                rand = random.random()
                if rand < 0.15:
                    wind_speed = 0
                elif rand < 0.40:
                    wind_speed = random.randint(1,10)
                elif rand < 0.40:
                    wind_speed = random.randint(11,20)
                else:
                    wind_speed = random.randint(21,30)

                cells[x][y] = Cell(self, x, y,
                    type_ = map[x][y],
                    temperature = random.randint(15, 27) if map[x][y] != "iceberg" else random.randint(-25, -6),
                    wind_speed = wind_speed,
                    wind_direction = random.choice(["N", "W"]),
                    airPollutionFactor = self.airPollutionFactor
                )

        return cells

    def _load_map(self, path):
        
        """"
        Loads the map structure from a text file. Each char represents a cell.
        Valid values are:
        L - land
        U - urban
        C - clouds
        F - forest
        S - sea
        I - iceberg
        """
        
        # length is +2 because we have an off-table layout of 1 cell length, for calculation reasons.
        map = [[0]*(self.length+2) for i in range(self.length+2)]

        with open(path, 'r') as f:
            for y in range(self.length):
                for x in range(self.length):
                    c = f.read(1)

                    while c not in ['L', 'U', 'C', 'F', 'S', 'I']:
                        c = f.read(1)

                    if c == "L":
                        map[x][y] = "land"
                    if c == "U":
                        map[x][y] = "urban"
                    if c == "C":
                        map[x][y] = "clouds"
                    if c == "F":
                        map[x][y] = "forest"
                    if c == "S":
                        map[x][y] = "sea"
                    if c == "I":
                        map[x][y] = "iceberg"

        return map


In [13]:
class Gui:
    def __init__(self, length, cell_size, map_file, refresh_rate=70, oneYear=365, airPollutionFactor=0.1):
        self.length = length
        self.cell_size = cell_size
        self.refresh_rate = refresh_rate  # in ms
        self.oneYear = oneYear

        # creates an empty self.length * self.length matrix
        self.items = [[0]*self.length for i in range(self.length)]

        self.table = Table(self.length, self.cell_size, map_file=map_file, airPollutionFactor=airPollutionFactor)
        self.root = tkinter.Tk()
        self.root.title("Biological computation- Global Warming")
        self.label = tkinter.Label(self.root)
        self.label.pack()
        self.canvas = tkinter.Canvas(self.root, height=self.length*self.cell_size, width=self.length*self.cell_size)
        self.canvas.pack()
        self.items = self.update_canvas(self.items)
        self.root.after(self.refresh_rate, self.refresh_screen)
        self.root.mainloop()

    def refresh_screen(self):
        self.table.runNewDay()
        

        if self.table.day >= self.oneYear:
            self.update_canvas(canvas_done=True, canvas_items=self.items)
            self.label.config(text="One year has passed")
        else:
            self.update_canvas(canvas_done=True, canvas_items=self.items)
            self.label.config(text="Day {}".format(self.table.day))
            self.root.after(self.refresh_rate, self.refresh_screen)
        
    def update_canvas(self, canvas_items, canvas_done=False):
        cell_items = self.table.cells

        if not canvas_done:
            for x in range(len(cell_items)):
                for y in range(len(cell_items)):
                    cell = cell_items[x][y]
                    cell_text = "{:.2f}".format(cell.airPollution)
                    cell_text = int(cell.temperature)
                    rectangle_id = self.canvas.create_rectangle(x*self.cell_size, y*self.cell_size, (x+1)*self.cell_size, (y+1)*self.cell_size, fill=cell.get_color())
                    text_id = self.canvas.create_text((x+0.5)*self.cell_size, (y+0.5)*self.cell_size, text=cell_text, font="Arial 8 bold")
                    canvas_items[x][y] = (rectangle_id, text_id)

            return canvas_items

        else:
            if canvas_items:
                for x in range(len(canvas_items)):
                    for y in range(len(canvas_items)):
                        cell = cell_items[x][y]
                        cell_text = "{:.2f}".format(cell.airPollution)
                        cell_text = int(cell.temperature)
                        (rectangle_id, text_id) = canvas_items[x][y]
                        self.canvas.itemconfig(rectangle_id, fill=cell.get_color())
                        self.canvas.itemconfig(text_id, text=cell_text)
            else:
                raise ValueError("No canvas_items given for re-iterating over canvas cells.")


In [ ]:
def mean(data):
    #Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/float(n)

def _ss(data):
    #Return sum of square deviations of sequence data.
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def stddev(data, ddof=0):
    #Calculates the population standard deviation
    #by default; specify ddof=1 to compute the sample
    #standard deviation.
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

if __name__ == '__main__':
    print("Hello! You are on the global warming and air pollution simulation.\n ")
    PullutionFactorInput = float(input("Please provide an air pollution factor, between 0 to 1 : \n(for a stable form, select up to 0.15. for a critical effect select 0.3 and above) \n\n"))
    runProgram = Gui(length=35, cell_size=22, refresh_rate=70, oneYear=365, map_file="WorldData.dat", airPollutionFactor = PullutionFactorInput)
    print("[\n\nTemperature]\t\tMax: {:.2f}\tMin: {:.2f}\tAvg: {:.2f}\tStd. Dev: {:.2f}".format(max(TEMPERATURE_STATS), min(TEMPERATURE_STATS), mean(TEMPERATURE_STATS), stddev(TEMPERATURE_STATS)))
    print("[Air Pollution]\t\tMax: {:.2f}\tMin: {:.2f}\tAvg: {:.2f}\tStd. Dev: {:.2f}".format(max(AIR_POLLUTION_STATS), min(AIR_POLLUTION_STATS), mean(AIR_POLLUTION_STATS), stddev(AIR_POLLUTION_STATS)))

    f = open('temperature.txt', 'w')
    for n in DAILY_TEMP_STATS: f.write("{}\n".format(n))
    f.close()

    f = open('air.txt', 'w')
    for n in DAILY_POLLUTION_STATS: f.write("{}\n".format(n))
    f.close()

   ## שאלה 1
    

א. נשים לב, שאם רק נסובב את הלוח 90 מעלות ימינה עם כיוון השעון נקבל את אותה המערכת בדיוק בעתיד.
בעצם, זה לא משנה אם קודם כל נסובב את הלוח ואז נפעיל את חוקי משחק החיים או להפך – קודם כל נפעיל את חוקי משחק החיים ואז נסובב את הלוח. בשני המקרים נגיע לאותה התוצאה בעתיד. 
זה נובע הסיבות הבאות:

כל תא מושפע אך ורק מהסביבה (8 השכנים) שלו(1) .

 הסיבוב לא משנה את שכני התא באופן מהותי, או יותר מדויק את השפעתם עליו.(2)

אזי, מ(1) ו- (2) מתחייב שהסיבוב לא משפיע על התא. לכן התוצאה של הפעלת חוקי משחק החיים בעתיד תהיה זהה

ב. אם נשנה את מודל האוטומט התאי למודל א-סינכרוני כמתואר בשאלה נקבל מודל שונה לחלוטין באופיו. המודל ישתנה בכך שכל תא יהיה תלוי בשאלה האם הוא זה ש"נבחר" שיפעלו עליו חוקי האוטומט עבור אותו פרק זמן, וזאת בניגוד למודל שלמדנו שבו התא תלוי רק בסביבתו.
כך נקבל במשחק החיים, למשל, שצורות (קבוצות תאים) אשר במודל ה"רגיל" משתנות בצורה כלשהי (גליידר, בלינקר וכו') עם הזמן ישתנו בצורה אחרת לגמרי, או לא ישתנו כלל במודל שמוצע כאן.
 אפשר לזקק את מהות ההבדל בין שני המודלים אולי בדטרמיניזם – המודל שנלמד הוא דטרמיניסטי באופיו והמודל החדש אינו דטרמיניסטי ותלוי ב"הגרלות" התאים שייבחרו באותו פרק זמן


# לינק לוידאו, בתא מתחת!

https://photos.app.goo.gl/HWjqaXQRbexXuVam7